In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import RunReportRequest

import os
scope = 'https://www.googleapis.com/auth/analytics.readonly'
key_file_location = "C:\Users\Owner\Downloads\json\ballosodeuk-ffa0e2802ed2.json"

# https://ga-dev-tools.google/ga4/query-explorer/

In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import RunReportRequest
from google.oauth2 import service_account
import pandas as pd

def fetch_ga4_data(property_id, dimension, times):
    # Initialize credentials with proper scope
    credentials = service_account.Credentials.from_service_account_file(
        'C:/Users/Owner/Downloads/json/ballosodeuk-ffa0e2802ed2.json',
        scopes=['https://www.googleapis.com/auth/analytics.readonly']
    )
    
    # Initialize client with credentials
    client = BetaAnalyticsDataClient(credentials=credentials)

    # GA4 데이터 요청
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=dimension,
        metrics=[
            {"name": "activeUsers"},
            {"name": "totalUsers"},
            ],
        date_ranges=[{"start_date": "2024-11-01", "end_date": "2024-12-31"}],
    )

    response = client.run_report(request)
    data = []
    # 결과 출력
    for row in response.rows:
        dimension_values = [dimension_value.value for dimension_value in row.dimension_values]
        metric_values = [metric_value.value for metric_value in row.metric_values]
        data.append(dimension_values + metric_values)
    
        # 컬럼명 설정
    dimension_names = [dimension.name for dimension in request.dimensions]
    metric_names = [metric.name for metric in request.metrics]
    columns = dimension_names + metric_names

    df = pd.DataFrame(data, columns=columns)

    if times == 'date':
        df[times] = pd.to_datetime(df[times])
    else:
        pass

    return df



if __name__ == "__main__":
    PROPERTY_ID = "318490867"
    return_dimension_dau = [
                {"name": "date"},
                {"name": "platform"},
                {"name": "newVsReturning"}
            ]
    total_dimension_dau = [
                {"name": "date"},
                {"name": "platform"},
            ]
    return_dimension_mau = [
                {"name": "month"},
                {"name": "platform"},
                {"name": "newVsReturning"}
            ]
    total_dimension_mau = [
                {"name": "month"},
                {"name": "platform"},
            ]

    return_dau_df = fetch_ga4_data(PROPERTY_ID,return_dimension_dau,'date')
    total_dau_df = fetch_ga4_data(PROPERTY_ID,total_dimension_dau,'date')
    return_mau_df = fetch_ga4_data(PROPERTY_ID,return_dimension_mau,'month')
    total_mau_df = fetch_ga4_data(PROPERTY_ID,total_dimension_mau,'month')

    return_dau_df = return_dau_df.query("platform != 'web'")
    return_dau_df.iloc[:,3:] = return_dau_df.iloc[:,3:].astype('int64')
    total_dau_df = total_dau_df.query("platform != 'web'")
    total_dau_df.iloc[:,2:] = total_dau_df.iloc[:,2:].astype('int64')
    return_mau_df = return_mau_df.query("platform != 'web'")
    return_mau_df.iloc[:,3:] = return_mau_df.iloc[:,3:].astype('int64')
    total_mau_df = total_mau_df.query("platform != 'web'")
    total_mau_df.iloc[:,2:] = total_mau_df.iloc[:,2:].astype('int64')
    
    
df_return_dau = return_dau_df.query('newVsReturning=="returning"').groupby('date').\
    agg({'activeUsers':'sum','totalUsers':'sum'})
df_total_dau = total_dau_df.groupby('date').\
    agg({'activeUsers':'sum','totalUsers':'sum'})
df_return_mau = return_mau_df.query('newVsReturning=="returning"').groupby('month').\
    agg({'activeUsers':'sum','totalUsers':'sum'})
df_total_mau = total_mau_df.groupby('month').\
    agg({'activeUsers':'sum','totalUsers':'sum'})

df_return_dau.rename(columns={'activeUsers':'returnActiveUsers','totalUsers':'returnTotalUsers'},inplace=True)
df_return_mau.rename(columns={'activeUsers':'returnActiveUsers','totalUsers':'returnTotalUsers'},inplace=True)


In [ ]:
df_merged_dau = df_total_dau.merge(df_return_dau,how='left',on='date')
df_merged_mau = df_total_mau.merge(df_return_mau,how='left',on='month')

df_merged_dau = df_merged_dau.groupby(pd.Grouper(freq='M')).agg({'totalUsers':'mean','returnTotalUsers':'mean'})
df_merged_mau = df_merged_mau.groupby('month').agg({'totalUsers':'mean','returnTotalUsers':'mean'})

In [ ]:
df_merged_dau.reset_index()

In [ ]:
df_merged_mau.reset_index()